In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV

In [2]:
train = pd.read_csv('train_dataset.csv')

In [3]:
test = pd.read_csv('test_dataset.csv')

In [34]:
'volume' in train.columns

False

In [4]:
train.head()

,id,location,fault_severity,severity_type 1,severity_type 2,severity_type 3,severity_type 4,severity_type 5,resource_type 1,resource_type 10,...,event_type 5,event_type 50,event_type 51,event_type 52,event_type 53,event_type 54,event_type 6,event_type 7,event_type 8,event_type 9
0,14121,location 118,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9320,location 91,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,14394,location 152,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,8218,location 931,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,14804,location 120,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
sample = pd.read_csv('sample_submission.csv')

In [28]:
sample.head()

,id,predict_0,predict_1,predict_2
0,11066,0,1,0
1,18000,0,1,0
2,16964,0,1,0
3,4795,0,1,0
4,3392,0,1,0


In [14]:
param = {'n_estimators': list(range(30, 80))}

In [15]:
rfc = RandomForestClassifier(n_jobs=-1)

In [16]:
gs = GridSearchCV(rfc, param)

In [17]:
gs.fit(train.loc[:, 'severity_type 1': 'event_type 9'], train.loc[:, 'fault_severity'])

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid={'n_estimators': [30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79]},
       pre_dispatch='2*n_jobs', refit=True, score_func=None, scoring=None,
       verbose=0)

In [18]:
gs.best_params_

{'n_estimators': 63}

In [6]:
rfc = RandomForestClassifier(n_estimators=63)

In [7]:
rfc.fit(train.loc[:, 'severity_type 1': 'event_type 9'], train.loc[:, 'fault_severity'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=63, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [8]:
result = rfc.predict(test.loc[:, 'severity_type 1': 'event_type 9'])

In [9]:
result.shape

(11171,)

In [10]:
rfc.classes_

array([0, 1, 2])

In [11]:
prob = rfc.predict_proba(test.loc[:, 'severity_type 1': 'event_type 9'])

In [12]:
prob.shape

(11171, 3)

In [13]:
submit = pd.DataFrame({
    'id': sample.loc[:, 'id'].values, 
    'predict_0': prob[:, 0],
    'predict_1': prob[:, 1],
    'predict_2': prob[:, 2]})

In [30]:
submit.head()

,id,predict_0,predict_1,predict_2
0,11066,1.000000,0.000000,0.000000
1,18000,0.095238,0.000000,0.904762
2,16964,1.000000,0.000000,0.000000
3,4795,0.746032,0.253968,0.000000
4,3392,0.191628,0.808372,0.000000


In [31]:
submit.to_csv('randomforest_after_grid_search.csv', index=False)

In [14]:
submit = pd.merge(submit, test.loc[:, ['id', 'location']], on='id', how='left')

In [15]:
submit.head()

,id,predict_0,predict_1,predict_2,location
0,11066,1.000000,0.000000,0.000000,location 481
1,18000,0.063492,0.000000,0.936508,location 962
2,16964,1.000000,0.000000,0.000000,location 491
3,4795,0.809524,0.190476,0.000000,location 532
4,3392,0.146121,0.853879,0.000000,location 600


In [16]:
fault_location = pd.read_csv('fault_severity_frac.csv', index_col=[0])

In [18]:
fault_location = fault_location.reset_index()

In [19]:
submit2 = pd.merge(submit, fault_location, on='location', how='left')

In [24]:
submit2.head()

,predict_0,predict_1,predict_2,location,0,1,2
id,,,,,,,
11066,1.000000,0.000000,0.000000,location 481,0.800000,0.150000,0.050000
18000,0.063492,0.000000,0.936508,location 962,0.288889,0.266667,0.444444
16964,1.000000,0.000000,0.000000,location 491,1.000000,0.000000,0.000000
4795,0.809524,0.190476,0.000000,location 532,0.666667,0.333333,0.000000
3392,0.146121,0.853879,0.000000,location 600,0.421875,0.125000,0.453125


In [21]:
submit2 = submit2.fillna(0)

In [22]:
submit2.shape

(11171, 8)

In [23]:
submit2 = submit2.set_index('id')

In [30]:
submit3 = submit2.loc[:, ['predict_0', 'predict_1', 'predict_2']].add(submit2.loc[:, ['0', '1', '2']].values) / 2

In [31]:
submit3.head()

,predict_0,predict_1,predict_2
id,,,
11066,0.900000,0.075000,0.025000
18000,0.176190,0.133333,0.690476
16964,1.000000,0.000000,0.000000
4795,0.738095,0.261905,0.000000
3392,0.283998,0.489439,0.226562


In [35]:
submit3.reset_index().to_csv('randomforest_location.csv', index=False)